#What is CVXPY?

CVXPY is a Python-embedded modeling language for convex optimization problems. CVXPY allows you to express your problem in a natural way. It automatically transforms the problem into standard form, calls a solver, and unpacks the results.

The code below solves a simple optimization problem in CVXPY:

In [3]:
from cvxpy import *

# Create two scalar variables.
x = Variable()
y = Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = Minimize(square(x - y))

# Form and solve problem.
prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:" + prob.status)
print("optimal value "+ str(prob.value))
print("optimal var " + str(x.value) + '  ' + str(y.value))

status:optimal
optimal value 1.0
optimal var 1.0  1.570086213240983e-22


The status, which was assigned a value "optimal" by the solve method, tells us the problem was solved successfully. The optimal value (basically 1 here) is the minimum value of the objective over all choices of variables that satisfy the constraints. The last thing printed gives values of x and y (basically 1 and 0 respectively) that achieve the optimal objective.

`prob.solve()` returns the optimal value and updates `prob.status`, `prob.value`, and the `value` field of all the variables in the problem.

##Namespace

The Python examples in this tutorial import CVXPY using the syntax ``from cvxpy import *``.
This is done to make the examples easier to read. But for production
code you should always import CVXPY as a namespace. For example,
``import cvxpy as cvx``.

Here's the code from the previous section with
CVXPY imported as a namespace:

In [ ]:
import cvxpy as cvx

# Create two scalar variables.
x = cvx.Variable()
y = cvx.Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = cvx.Minimize(cvx.square(x - y))

# Form and solve problem.
prob = cvx.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:" + prob.status)
print("optimal value "+ str(prob.value))
print("optimal var " + str(x.value) + '  ' + str(y.value))

Nonetheless we have constructed CVXPY so that using ``from cvxpy import *`` is generally safe for short scripts. The biggest catch is that the built-in ``max`` and ``min`` cannot be used on CVXPY expressions. Instead use the CVXPY [functions](/functions) ``maximum``, ``max``, ``minimum``, or ``min``.

##Changing the Problem

After you create a problem object, you can still modify the objective and constraints.

In [ ]:
# Replace the objective.
prob = cvx.Problem(cvx.Maximize(x + y), prob.constraints)
print("optimal value", prob.solve())

# Replace the constraint (x + y == 1).
prob.constraints[0] = (x + y <= 3)
print("optimal value " + str(prob.solve()))

##Infeasible and Unbounded Problems

If a problem is infeasible or unbounded, the status field will be set to "infeasible" or "unbounded", respectively. The value fields of the problem variables are not updated.

In [1]:
from cvxpy import *

x = Variable()

# An infeasible problem.
prob = Problem(Minimize(x), [x >= 1, x <= 0])
prob.solve()
print("status:" + prob.status)
print("optimal value "+ str(prob.value))

# An unbounded problem.
prob = Problem(Minimize(x))
prob.solve()
print("status:" + prob.status)
print("optimal value "+ str(prob.value))

status:infeasible
optimal value inf
status:unbounded
optimal value -inf


Notice that for a minimization problem the optimal value is `inf` if infeasible and `-inf` if unbounded. For maximization problems the opposite is true.

If the solver called by CVXPY fails to solve the problem, the problem status is set to "solver_error" and the optimal value is `None`. See the discussion of [Solvers](#solvers) for details.

CVXPY provides the constants `OPTIMAL`, `INFEASIBLE`, `UNBOUNDED`, and `SOLVER_ERROR` as aliases for the different status strings.

##Vectors and Matrices

Variables can be scalars, vectors, or matrices.

In [ ]:
# A scalar variable.
a = Variable()

# Column vector variable of length 5.
x = Variable(5)

# Matrix variable with shape (5, 1).
x = Variable((5, 1))

# Matrix variable with 4 rows and 7 columns.
A = Variable((4, 7))

You can use your numeric library of choice to construct matrix and vector constants. For instance, if `x` is a CVXPY Variable in the expression `A*x + b`, `A` and `b` could be Numpy ndarrays, SciPy sparse matrices, etc. `A` and `b` could even be different types.

Currently the following types may be used as constants:

* Numpy ndarrays
* Numpy matrices
* SciPy sparse matrices
* CVXOPT dense matrices
* CVXOPT sparse matrices

Here's an example of a CVXPY problem with vectors and matrices:

In [ ]:
# Solves a bounded least-squares problem.

from cvxpy import *
import numpy

# Problem data.
m = 10
n = 5
numpy.random.seed(1)
A = numpy.random.randn(m, n)
b = numpy.random.randn(m)

# Construct the problem.
x = Variable(n)
objective = Minimize(sum(square(A*x - b)))
constraints = [0 <= x, x <= 1]
prob = Problem(objective, constraints)

print("Optimal value " + str(prob.solve()))
print("Optimal var " + str(x.value)) # A numpy matrix.